In [1]:
from pytube import YouTube
from moviepy.editor import *
import mutagen
from mutagen.mp3 import MP3
from mutagen.easyid3 import EasyID3
from pydub import AudioSegment
import os
import sys
import re
from utils import get_url, extract_string, strip_string, parse_tracklist

In [5]:
descriptions = [
    YouTube("https://www.youtube.com/watch?v=Y0Oa4Lp5fLE&t=1s").description,
    YouTube("https://www.youtube.com/watch?v=ovIh2WVhbPs").description,
    YouTube("https://www.youtube.com/watch?v=OGTgkRCp89M").description,
    YouTube("https://www.youtube.com/watch?v=rqFpbo_K2Eg").description,
    YouTube("https://www.youtube.com/watch?v=DuIGJiqCb8w").description,
    YouTube("https://www.youtube.com/watch?v=ProvVFrF6b8").description,
    YouTube("https://www.youtube.com/watch?v=MEW4GczA5PM").description,
    YouTube("https://www.youtube.com/watch?v=3u-4fxKX8as").description,
    YouTube("https://www.youtube.com/watch?v=Z3m7HXeiHpg").description,
    YouTube("https://www.youtube.com/watch?v=ElrSLxxGSvw").description,
    YouTube("https://www.youtube.com/watch?v=skSx-ECpzmw").description,
    YouTube("https://www.youtube.com/watch?v=u5PG7aRYS1k&t=1s").description,
    YouTube("https://www.youtube.com/watch?v=ZJSuKb8YAYI").description,
    YouTube("https://www.youtube.com/watch?v=6hIgBEXuQD8").description,
    YouTube("https://www.youtube.com/watch?v=LL998ajnjN4").description,
    YouTube("https://www.youtube.com/watch?v=l0vrsO3_HpU").description,
]

In [7]:
def ts_decompose(ts):
    stamps = ts.split(":")
    if len(stamps) == 3:
        hours = int(stamps[0])
        mnts = int(stamps[1])
        secs = int(stamps[2])
        mls = hours*60*60*1000 + mnts*60*1000 + secs*1000
    else:
        mnts = int(stamps[1])
        secs = int(stamps[0])
        mls = hours*60*60*1000 + mnts*60*1000 + secs*1000
        
    print(hours, mnts, secs)
    print(mls)
    
ts_decompose("00:12:01")

0 12 1
721000


In [13]:
for d in descriptions[:]:
    print("+++++++++++++++++++++++++++")
    metadata = parse_tracklist(d)
    print(metadata)
    for i in metadata:
        print(i)

+++++++++++++++++++++++++++
[]
+++++++++++++++++++++++++++
[[1, 0, 'Concerning the UFO Sighting Near Highland, Illinois'], [2, 129000, "The Black Hawk War, or, How to Demolish an Entire Civilization and Still Feel Good About Yourself in the Morning, or, We Apologize for the Inconvenience but You're Going to Have to Leave Now, or, 'I Have Fought the Big Knives and Will Continue to Fight Them Until They Are Off Our Lands!'"], [3, 263000, "Come On! Feel the Illinoise! (Part I  The World's Columbian Exposition   Part II  Carl Sandburg Visits Me in a Dream)"], [4, 668000, 'John Wayne Gacy, Jr'], [5, 868000, 'Jacksonville'], [6, 1192000, 'A Short Reprise for Mary Todd, Who Went Insane, but for Very Good Reasons'], [7, 1240000, 'Decatur, or, Round of Applause for Your Stepmother!'], [8, 1424000, "One Last 'Whoo Hoo!' for the Pullman"], [9, 1430000, 'Chicago'], [10, 1795000, 'Casimir Pulaski Day'], [11, 2148000, 'To the Workers of the Rock River Valley Region, I Have an Idea Concerning Your Pr

In [12]:
class ParsingError(Exception):
    pass

def ts_decompose(ts):
    stamps = ts.split(":")
    if len(stamps) == 3:
        hours = int(stamps[0])
        mnts = int(stamps[1])
        secs = int(stamps[2])
        mls = hours*60*60*1000 + mnts*60*1000 + secs*1000
    else:
        mnts = int(stamps[0])
        secs = int(stamps[1])
        mls = mnts*60*1000 + secs*1000
    
    return mls

def strip_string(line):
    flags = ["-", "–", "|", ":"]
    for flag in flags:
        line = line.replace(flag, " ")
    line = line.strip().strip(".").strip()
    return line

def parse_tracklist(description):
    # Initialize local variables
    ts_format = ["[0-9]:[0-9][0-9]:[0-9][0-9]", "[0-9]:[0-9][0-9]"]
    tracknum_format = ["[ABCDEFGH][0-9]", "[0-9]\.", "[0-9]\ ", "[0-9][0-9]\.", "[0-9][0-9]\ "]
    TS = {}
    TN = {}
    titles = {}
    span_idx = []
    desc = description.split("\n")
    for idx, line in zip(range(len(desc)), desc):
        # Init
        new_line = None
        ts_search = None
        tracknum = None
        
        # Search for a timestamp in the current line
        for f in ts_format:
            if not ts_search:
                ts_search = re.search(f, line)
        
        
        if ts_search:
            # If timestamp is detected, extract it from the line and create a new line without it
            span = ts_search.span()
            span_idx.append(span[1])
            if span[0] == 0:
                span = [1, span[1]]
            #print(line[span[0]-1: span[1]+1])
            TS[idx] = str(line[span[0]-1: span[1]]).replace(" ", "0")
            new_line = line[:span[0]-1] + line[span[1]+1:]
        else:
            # If no timestamp is detected, pass on to the next line
            continue
        
        # Search for a track number in the new line
        for f in tracknum_format:
            search = re.search(f, new_line)
            if search:
                #print(f)
                tracknum = search
                break
        
        # If track number is found, extract it from the line
        if tracknum:
            span_track = tracknum.span()
            #print(new_line[span_track[0]: span_track[1]])
            TN[idx] = new_line[span_track[0]: span_track[1]]
        
        # Parse the title from the line depending on track number and timestamp positions
        if not tracknum:
            title = strip_string(new_line)
            titles[idx] = title
        elif span_track[0] < (len(new_line)/2):
            title = strip_string(new_line[span_track[1]:])
            titles[idx] = title
        else:
            title = strip_string(new_line[:span_track[0]])
            titles[idx] = title
    
    # Compile collected metadata and return final value
    metadata = []
    if len(titles) == len(TS):
        for idx, k in zip(range(1, len(titles)+1), titles.keys()):
            stamp = ts_decompose(TS[k])
            metadata.append([idx, stamp, titles[k]])
            #print([idx, TS[k], titles[k]])
        return metadata
    elif len(titles) == len(TS)+1:
        # This case is for when no "00:00" timestamp is provided
        for idx, k in zip(range(1, len(titles)+1), titles.keys()):
            if idx == 1:
                metadata.append([idx, 0, titles[k]])
                continue
            else:
                stamp = ts_decompose(TS[k])
                metadata.append([idx, stamp, titles[k]])
        return metadata
    else:
        raise ParsingError("Could not match timestamps with titles. {x} timestamps found, {y} titles found.".format(x = len(TS), y = len(titles)))
        return []

if track num and timestamp => title is in between
if no num and timestamp is at the beginning of string (index 1 to 4) => title is after 
opposite is true if timestamp at the end

strip title from all possible separators : "-", "–", "|", "."